<a href="https://colab.research.google.com/github/chuckfinca/arithmephi/blob/main/arithmephi_inference_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.0
!pip install pyarrow==14.0.2
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install einops datasets bitsandbytes accelerate peft flash_attn num2words

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3btfmzn5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3btfmzn5
  Resolved https://github.com/huggingface/transformers to commit 0290ec19c901adc0f1230ebdccad11c40af026f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661022 sha256=d0b878d9104cec9737e021c05e85e43fbdb4a6bf28c176d0c728813afcbaae15
  Stored in directory: /tmp/pip-ephem-wheel-cache-_1p5hzet/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━

In [2]:
# install git-lfs to interact with the repository
!sudo apt-get update
!sudo apt-get install git-lfs
# install transformers (not needed since it is installed by default in the container)
# !pip install transformers#[sklearn,sentencepiece,audio,vision]

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,561 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,951 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/u

In [5]:
# from huggingface_hub import login
# login()

In [6]:
# !git config --global credential.helper store

In [3]:
!git lfs install
!git clone https://huggingface.co/chuckfinca/arithmephi

Git LFS initialized.
Cloning into 'arithmephi'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 51 (delta 18), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (51/51), 30.34 KiB | 2.17 MiB/s, done.


In [4]:
!cd arithmephi && touch handler.py

In [2]:
import flash_attn, torch, locale, pandas, time, random, string, num2words
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

In [3]:
class EndpointHandler():
    def __init__(self, path=""):
        torch.set_default_device("cuda")
        self.model = AutoModelForCausalLM.from_pretrained("chuckfinca/arithmephi", torch_dtype="auto", trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

    def __call__(self, data: str) -> str:
        inputs = self.tokenizer(data, return_tensors="pt", return_attention_mask=False)
        outputs = self.model.generate(**inputs, max_length=len(inputs[0]) + 8, pad_token_id=self.tokenizer.eos_token_id)
        text = self.tokenizer.batch_decode(outputs)[0]
        return text

In [22]:
# !ls

adapter_model.safetensors  config.json	handler.py  README.md


In [31]:
# import importlib.util
# print(importlib.util.find_spec("safetensors") is not None)

True


In [4]:
# import os

# os.listdir()
# from pipeline import PreTrainedPipeline

# init handler
my_handler = EndpointHandler(path="chuckfinca/arithmephi") #"microsoft/phi-2")#

# prepare sample payload
request = "I am quite excited how this will turn out"

# test the handler
%timeit my_handler(request)

adapter_config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
I am quite excited how this will turn out
I am quite excited how this will turn out. I am sure it will be a
697 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
cd arithmephi

/content/arithmephi
